### 1. a) Create a **_Term Document Incidence Matrix_** for the given documents

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [3]:
def termDocMatrix(docs, n):
    terms = []
    for i in range(n):
        terms.extend(docs[i].split())
    terms = sorted(list(set(terms)))

    mat = np.zeros((len(terms), n), dtype=int)

    for i in range(len(terms)):
        for j in range(len(docs)):
            if terms[i] in docs[j]:
                mat[i, j] = 1

    data = {"Terms": terms}
    for i in range(n):
        data[f"Doc{i+1}"] = mat[:, i]
    df = pd.DataFrame(data)
    display(HTML(df.to_html(index=False)))
    return (terms, mat)

In [11]:
n = int(input("Enter the number of documents: "))
docs = []
print(f"Enter the contents of {n} documents: ")
for i in range(n):
    docs.append(input(f"\tDocument {i+1}: "))
terms, mat = termDocMatrix(docs, n)

Enter the number of documents:  4


Enter the contents of 4 documents: 


	Document 1:  breakthrough drug for schizophrenia
	Document 2:  new schizophrenia drug
	Document 3:  new approach for treatment of schizophrenia
	Document 4:  new hopes for schizophrenia patients


Terms,Doc1,Doc2,Doc3,Doc4
approach,0,0,1,0
breakthrough,1,0,0,0
drug,1,1,0,0
for,1,0,1,1
hopes,0,0,0,1
new,0,1,1,1
of,0,0,1,0
patients,0,0,0,1
schizophrenia,1,1,1,1
treatment,0,0,1,0


## 1. b) Process boolean queries

In [4]:
def binOp(vals, op, n):
    ans = []
    if op == 'NOT':
        ans = [1 if vals[0][i]==0 else 0 for i in range(n)]
    elif op == 'AND':
        ans = [vals[0][i] & vals[1][i] for i in range(n)]
    elif op == 'OR':
        ans = [vals[0][i] | vals[1][i] for i in range(n)]
    return ans

In [5]:
def andor(query, n):
    vals = []
    flag = 0
    for i in range(len(query)):
        if query[i] not in ["AND", "OR"]:
            vals.append(query[i])
            if flag == 1:
                vals = binOp(vals, query[i-1], n)
                flag = 0
        else:
            flag = 1
    return vals

In [6]:
def processQ(query, n):
    sIdx = 0
    ans = []
    while sIdx < len(query):
        sIdx = query.index('(') if '(' in query else -1
        if sIdx == -1:
            break
        for i in range(sIdx, len(query)):
            idx1 = query.index('(', sIdx) if '(' in query else -1
            idx2 = query.index(')', sIdx) if ')' in query else -1
            if idx1 > idx2: break
            sIdx = idx1 if idx1 != -1 else sIdx
        eIdx = query.index(')', sIdx)
        ans = processQ(query[sIdx+1:eIdx], n)
        del query[sIdx+1:eIdx+1]
        query[sIdx] = ans
    idx = 0
    while idx < len(query):
        idx = query.index("NOT") if "NOT" in query else -1
        if idx == -1:
            break
        ans = binOp([query[idx+1]], "NOT", n)
        del query[idx+1]
        query[idx] = ans
    query = andor(query, n)
    return query

In [10]:
n = int(input("Enter the number of documents: "))
docs = []
print(f"Enter the contents of {n} documents: ")
for i in range(n):
    docs.append(input(f"\tDocument {i+1}: "))
query = input("Enter the boolean query:").split()
# flag = 0
terms, mat = termDocMatrix(docs, n)
for i in range(len(query)):
    if query[i] in terms:
        query[i] = list(mat[terms.index(query[i]), :])
    elif query[i] in ["(", ")", "NOT", "AND", "OR"]:
        continue
    else:
        query[i] = [0 for i in range(n)]
        
ans = processQ(query, 4)
print("The request query is processed and the documents are: ")
for i in range(n):
    if ans[i] == 1:
        print(f"Document{i+1}: ", docs[i])

Enter the number of documents:  4


Enter the contents of 4 documents: 


	Document 1:  breakthrough drug for schizophrenia
	Document 2:  new schizophrenia drug
	Document 3:  new approach for treatment of schizophrenia
	Document 4:  new hopes for schizophrenia patients
Enter the boolean query: sxchi AND new


Terms,Doc1,Doc2,Doc3,Doc4
approach,0,0,1,0
breakthrough,1,0,0,0
drug,1,1,0,0
for,1,0,1,1
hopes,0,0,0,1
new,0,1,1,1
of,0,0,1,0
patients,0,0,0,1
schizophrenia,1,1,1,1
treatment,0,0,1,0


The request query is processed and the documents are: 
